# Trabalho 2 - Computação Gráfica
## Cenário 3D
---
**Professor**: Alaor Cervati Neto

**Alunos**: 
- Adrio Oliveira Alves - 11796830
- Gabriel Freitas Ximenes de Vasconcelos - 11819084
- João Victor Sene Araújo - 11796382
- Raíssa Torres Barreira - 11796336
- Thiago Henrique Santos Cardoso - 11796594
- Victor Paulo Cruz Lutes - 11795512

Bibliotecas Utilizadas no trabalho:

In [1]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import glm
import math
from PIL import Image

### Inicializando janela

In [2]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
largura = 1600
altura = 1000
window = glfw.create_window(largura, altura, "Cenário 3D", None, None)
glfw.make_context_current(window)

### GLSL para *Vertex Shader*
No Pipeline programável, podemos interagir com *Vertex Shaders*.

No código abaixo, estamos fazendo o seguinte:
* Definindo uma variável chamada `position` do tipo `vec3`.
* Definindo uma variável chamada `texture_coord` do tipo `vec2`.
* Definindo uma variável chamada `normals` do tipo `vec3`.
* Definindo uma variável de saída `out_texture` do tipo `vec2`.
* Definindo uma variável de saída `out_fragPos` do tipo `vec3`.
* Definindo uma variável de saída `out_normal` do tipo `vec3`.
* Definindo matrizes `model`, `view`, e `projection` que acumulam transformações geométricas 3D e permitem navegação no cenário.
* `void main()` é o ponto de entrada do nosso programa (função principal).
* `gl_Position` é uma variável especial do `GLSL`. Variáveis que começam com `gl_` são desse tipo. Neste caso, determina a posição de um vértice. Observe que todo vértice tem $4$ coordenadas, por isso combinamos nossa variável `vec3` com uma variável `vec4`. Além disso, modificamos nosso vetor com base nas transformações `model`, `view`, e `projection`.
* `out_texture` é uma variável que retorna as coordenadas de texturas normalizadas.
* `out_fragPos` é uma variável que retorna as coordenadas dos fragmentos (manipuladas pelo usuário).
* `out_texture` é uma variável que retorna as coordenadas dos vetores normais (manipuladas pelo usuário).

In [3]:
vertex_code = """
        attribute vec3 position;
        attribute vec2 texture_coord;
        attribute vec3 normals;
        
        varying vec2 out_texture;
        varying vec3 out_fragPos;
        varying vec3 out_normal;
                
        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 projection;        
        
        void main(){
            gl_Position = projection * view * model * vec4(position,1.0);
            out_texture = vec2(texture_coord);
            out_fragPos = vec3(model * vec4(position, 1.0));
            out_normal = vec3(model * vec4(normals, 1.0));            
        }
        """

### GLSL para *Fragment Shader*
No Pipeline programável, podemos interagir com *Fragment Shaders*.

No código abaixo, estamos fazendo o seguinte:
* Definindo uma variável `lightPos`, do tipo `vec3`, que define a posição da fonte de luz.
* Definindo uma variável `lightColor`, do tipo `vec3`, para a intensidade da fonte de luz.
* Definindo uma variável `viewPos`, do tipo `vec3`, que define a posição do observador.
* Definindo as variáves `ka`, `kd`, `ks`, e `ns`, do tipo `float`, coeficientes e expoente de reflexão do objeto.
* Definindo as variáveis `out_texture`, `out_normal`, e `out_fragPos`, recebidas do *Vertex Shader*.
* Definindo uma variável do tipo `Sampler2D` chamada `samplerTexture`, que receberá os texeis da imagem.
* `void main()` é o ponto de entrada do nosso programa (função principal).
* `ambient` define a componente da iluminação ambiente do objeto.
* `diffuse` define a componente da reflexão difusa do objeto.
* `specular` define a componente da reflexão especular do objeto.
* `texture` é o resultado da operação de mapeamento das coordenadas de `out_texture` em `samplerTexture`.
* `result` é a resultante das componentes de iluminação a ser aplicada sobre a textura.
* `gl_FragColor` é uma variável especial do `GLSL`. Variáveis que começam com `gl_` são desse tipo. Neste caso, determina a cor de um fragmento de acordo com os valores de cor de uma imagem afetada pela iluminação.

In [4]:
fragment_code = """

        // parametro com a cor da(s) fonte(s) de iluminacao
        uniform vec3 lightPos; // define coordenadas de posicao da luz
        vec3 lightColor = vec3(5.0, 5.0, 5.0);
        
        // parametros da iluminacao ambiente e difusa
        uniform float ka; // coeficiente de reflexao ambiente
        uniform float kd; // coeficiente de reflexao difusa
        
        // parametros da iluminacao especular
        uniform vec3 viewPos; // define coordenadas com a posicao da camera/observador
        uniform float ks; // coeficiente de reflexao especular
        uniform float ns; // expoente de reflexao especular

        // parametros recebidos do vertex shader
        varying vec2 out_texture; // recebido do vertex shader
        varying vec3 out_normal; // recebido do vertex shader
        varying vec3 out_fragPos; // recebido do vertex shader
        uniform sampler2D samplerTexture;
        
        void main(){
        
            // calculando reflexao ambiente
            vec3 ambient = ka * lightColor;             
        
            // calculando reflexao difusa
            vec3 norm = normalize(out_normal); // normaliza vetores perpendiculares
            vec3 lightDir = normalize(lightPos - out_fragPos); // direcao da luz
            float diff = max(dot(norm, lightDir), 0.0); // verifica limite angular (entre 0 e 90)
            vec3 diffuse = kd * diff * lightColor; // iluminacao difusa
            
            // calculando reflexao especular
            vec3 viewDir = normalize(viewPos - out_fragPos); // direcao do observador/camera
            vec3 reflectDir = normalize(reflect(-lightDir, norm)); // direcao da reflexao
            float spec = pow(max(dot(viewDir, reflectDir), 0.0), ns);
            vec3 specular = ks * spec * lightColor;             
            
            // aplicando o modelo de iluminacao
            vec4 texture = texture2D(samplerTexture, out_texture);
            vec4 result = vec4((ambient + diffuse + specular),1.0) * texture; // aplica iluminacao
            gl_FragColor = result;

        }
        """

### Requisitando slot para a GPU para nossos programas *Vertex* e *Fragment Shaders*

In [5]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

### Associando nosso código-fonte aos slots solicitados

In [6]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o *Vertex Shader*
Exibe erro no Vertex Shader caso ocorra.

In [7]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

### Compilando o *Fragment Shader*
Exibe erro no Fragment Shader caso ocorra.

In [8]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilados ao programa principal

In [9]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)

### Linkagem do programa

In [10]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Preparando dados para enviar a GPU
Nesse momento, compilamos nossos *Vertex* e *Fragment Shaders* para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.
### Carregando Modelos (vértices e texturas) a partir de Arquivos
A função abaixo carrega modelos a partir de arquivos no formato *WaveFront*.

Para saber mais sobre o modelo, acesse: https://en.wikipedia.org/wiki/Wavefront_.obj_file

Nos slides, descrevemos o funcionamento desse formato.

In [11]:
def load_model_from_file(filename):
    """Loads a Wavefront OBJ file. """
    objects = {}
    vertices = []
    normals = []
    texture_coords = []
    faces = []

    material = None

    # abre o arquivo obj para leitura
    for line in open(filename, "r"): ## para cada linha do arquivo .obj
        if line.startswith('#'): continue ## ignora comentarios
        values = line.split() # quebra a linha por espaço
        if not values: continue


        ### recuperando vertices
        if values[0] == 'v':
            vertices.append(values[1:4])

        ### recuperando vertices
        if values[0] == 'vn':
            normals.append(values[1:4])

        ### recuperando coordenadas de textura
        elif values[0] == 'vt':
            texture_coords.append(values[1:3])

        ### recuperando faces 
        elif values[0] in ('usemtl', 'usemat'):
            material = values[1]
        elif values[0] == 'f':
            face = []
            face_texture = []
            face_normals = []
            for v in values[1:]:
                w = v.split('/')
                face.append(int(w[0]))
                face_normals.append(int(w[2]))
                if len(w) >= 2 and len(w[1]) > 0:
                    face_texture.append(int(w[1]))
                else:
                    face_texture.append(0)

            faces.append((face, face_texture, face_normals, material))

    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['faces'] = faces
    model['normals'] = normals

    return model

glEnable(GL_TEXTURE_2D)
qtd_texturas = 10
textures = glGenTextures(qtd_texturas)

def load_texture_from_file(texture_id, img_textura):
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    img = Image.open(img_textura)
    img_width = img.size[0]
    img_height = img.size[1]
    image_data = img.tobytes("raw", "RGB", 0, -1)
    #image_data = np.array(list(img.getdata()), np.uint8)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, img_width, img_height, 0, GL_RGB, GL_UNSIGNED_BYTE, image_data)

### A lista abaixo armazena todos os vértices carregados dos arquivos

In [12]:
vertices_list = []    
normals_list = []    
textures_coord_list = []

### Vamos carregar cada modelo e definir funções para desenhá-los

In [13]:
def load_model(model_path, texture_path, texture_id_index):
    """
        Carrega um modelo 3D do tipo .obj que possua todas as faces triângulares e sua respectiva textura
        
        Parâmetros:
            model_path = caminho do modelo 3D
            texture_path = caminho da textura do modelo 3D
            texture_id_index = id da textura carregada
        
        Retorno:
            Lista com os os componentes:
                [
                    initial_index = indice do vertice inicial
                    final_index = indice do vertice final
                    texture_id_index = id da textura carregada
                ]
    """
    
    modelo = load_model_from_file(model_path)
    
    initial_index = len(vertices_list)
    
    # Inserindo vertices do modelo no vetor de vertices
    for face in modelo['faces']:
        for vertice_id in face[0]:
            vertices_list.append( modelo['vertices'][vertice_id-1] )
        for texture_id in face[1]:
            textures_coord_list.append( modelo['texture'][texture_id-1] )
        for normal_id in face[2]:
            normals_list.append( modelo['normals'][normal_id-1] )
            
    final_index = len(vertices_list)
    
    # Inserindo coordenadas de textura do modelo no vetor de texturas
    # Carregando textura equivalente e definindo um id (buffer): use um id por textura!
    load_texture_from_file(texture_id_index,texture_path)
    
    return [initial_index, final_index, texture_id_index]

### Agora, vamos abrir os modelos e guardar os dados em uma lista para exibi-los na tela posteriormente

In [14]:
texture_next_id = 0
models_indexes = []

models_indexes.append(load_model('Models/sky.obj', 'Textures/sky.jpg', texture_next_id))
texture_next_id += 1

models_indexes.append(load_model('Models/sun.obj', 'Textures/sun.png', texture_next_id))
texture_next_id += 1

models_indexes.append(load_model('Models/dirt.obj', 'Textures/dirt.jpg', texture_next_id))
texture_next_id += 1

models_indexes.append(load_model('Models/grass.obj', 'Textures/grass.jpg', texture_next_id))
texture_next_id += 1

models_indexes.append(load_model('Models/house.obj', 'Textures/house.png', texture_next_id))
texture_next_id += 1

models_indexes.append(load_model('Models/couch.obj', 'Textures/couch.jpg', texture_next_id))
texture_next_id += 1

models_indexes.append(load_model('Models/table.obj', 'Textures/table.png', texture_next_id))
texture_next_id += 1

models_indexes.append(load_model('Models/tv.obj', 'Textures/tv.jpg', texture_next_id))
texture_next_id += 1

models_indexes.append(load_model('Models/fence.obj', 'Textures/fence.jpg', texture_next_id))
texture_next_id += 1

models_indexes.append(load_model('Models/dog_house.obj', 'Textures/dog_house.jpg', texture_next_id))
texture_next_id += 1

models_indexes.append(load_model('Models/dachshund.obj', 'Textures/dachshund.png', texture_next_id))
texture_next_id += 1

### Para enviar nossos dados da CPU para a GPU, precisamos requisitar slots.
Agora requisitaremos três slots.
* Um para enviar coordenadas dos vértices.
* Um para enviar coordenadas de texturas.
* Um para enviar coordenadas de normals para iluminação.

In [15]:
# Request a buffer slot from GPU
buffer = glGenBuffers(3)

###  Enviando coordenadas de vértices para a GPU

In [16]:
vertices = np.zeros(len(vertices_list), [("position", np.float32, 3)])
vertices['position'] = vertices_list

# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[0])
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)

###  Enviando coordenadas de textura para a GPU

In [17]:
textures = np.zeros(len(textures_coord_list), [("position", np.float32, 2)]) # duas coordenadas
textures['position'] = textures_coord_list

# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)

###  Enviando dados de Iluminação a GPU
#### Dados de iluminação: vetores normais

In [18]:
normals = np.zeros(len(normals_list), [("position", np.float32, 3)]) # três coordenadas
normals['position'] = normals_list

# Upload coordenadas normals de cada vertice
glBindBuffer(GL_ARRAY_BUFFER, buffer[2])
glBufferData(GL_ARRAY_BUFFER, normals.nbytes, normals, GL_STATIC_DRAW)
stride = normals.strides[0]
offset = ctypes.c_void_p(0)
loc_normals_coord = glGetAttribLocation(program, "normals")
glEnableVertexAttribArray(loc_normals_coord)
glVertexAttribPointer(loc_normals_coord, 3, GL_FLOAT, False, stride, offset)

### Desenhando nossos modelos
* Cada modelo tem uma *Model* para posicioná-lo no mundo.
* É necessário saber qual a posição inicial e total de vértices de cada modelo.
* É necessário indicar qual o `id` da textura do modelo.

In [19]:
def draw_object(model_index, angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z, ka=0.1, kd=0.1, ks=0.9, ns=32, emit_light=False):
    """
        Desenha um modelo já carregado, parâmetros de transformações 3D e influência da iluminação
        
        Parâmetros:
            model_index = lista com 3 atributos: initial_index, final_index, texture_id
            angle = ângulo de rotação
            r_x, r_y, r_z = coordenadas de rotação
            t_x, t_y, t_z = coordenadas de translação (posição)
            s_x, s_y, s_z = coordenadas de escala
            ka = coeficiente de reflexao ambiente do modelo
            kd = coeficiente de reflexao difusa do modelo
            ks = coeficiente de reflexao especular do modelo
            ns = expoente de reflexao especular
            emit_light = booleano que indica se o objeto emite luz
        
        Retorno:
            Vazio
    """
    
    # Aplica a matriz model
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    loc_ka = glGetUniformLocation(program, "ka") # Recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_ka, ka) # Envia ka pra gpu
    
    loc_kd = glGetUniformLocation(program, "kd") # Recuperando localizacao da variavel kd na GPU
    glUniform1f(loc_kd, kd) # Envia kd pra gpu    
    
    loc_ks = glGetUniformLocation(program, "ks") # Recuperando localizacao da variavel ks na GPU
    glUniform1f(loc_ks, ks) # Envia ks pra gpu        
    
    loc_ns = glGetUniformLocation(program, "ns") # Recuperando localizacao da variavel ns na GPU
    glUniform1f(loc_ns, ns) # Envia ns pra gpu
    
    if (emit_light == True):
        loc_light_pos = glGetUniformLocation(program, "lightPos") # Recuperando localizacao da variavel lightPos na GPU
        glUniform3f(loc_light_pos, t_x, t_y, t_z) # Posicao da fonte de luz
    
    # Define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, model_index[2])
    
    # Desenha o modelo
    glDrawArrays(GL_TRIANGLES, model_index[0], model_index[1] - model_index[0]) # Renderizando

### Eventos para modificar a posição da câmera.
* As teclas `A`, `S`, `D`, e `W` são usadas para a movimentação no espaço tridimensional.
* `Left Shift` aumenta a velocidade de movimentação, enquanto `Left Ctrl` diminui.
* A posição do mouse direciona a câmera.
* As setinhas `Esquerda` e `Direita` mudam o parâmetro largura da matriz projection.
* As setinhas `Cima` e `Baixo` mudam o parâmetro altura da matriz projection

In [20]:
cameraPos   = glm.vec3(0.0,  10.0,  0.0);
cameraFront = glm.vec3(0.0,  0.0, -1.0);
cameraUp    = glm.vec3(0.0,  1.0,  0.0);
cameraSpeed = 0.05

polygonal_mode = False

def key_event(window,key,scancode,action,mods):
    global cameraPos, cameraFront, cameraUp, polygonal_mode
    global altura, largura
    global cameraSpeed
    
    if key == 87 and (action==1 or action==2): # tecla W
        cameraPos += cameraSpeed * cameraFront
    
    if key == 83 and (action==1 or action==2): # tecla S
        cameraPos -= cameraSpeed * cameraFront
    
    if key == 65 and (action==1 or action==2): # tecla A
        cameraPos -= glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if key == 68 and (action==1 or action==2): # tecla D
        cameraPos += glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if key == 265 and (action==1 or action==2): # tecla UP
        altura += 1    
        
    if key == 264 and (action==1 or action==2): # tecla DOWN
        altura -= 1  
        
    if key == 262 and (action==1 or action==2): # tecla RIGHT
        largura += 1 
        
    if key == 263 and (action==1 or action==2): # tecla LEFT
        largura -= 1 
        
    if key == 340 and (action==1 or action==2): # tecla Shift
        cameraSpeed += 0.05
        
    if key == 341 and (action==1 or action==2): # tecla Ctrl
        cameraSpeed -= 0.05
    
    if key == 80 and action==1 and polygonal_mode==True:
        polygonal_mode=False
    else:
        if key == 80 and action==1 and polygonal_mode==False:
            polygonal_mode=True
        
firstMouse = True
yaw = -90.0 
pitch = 0.0
lastX =  largura/2
lastY =  altura/2

def mouse_event(window, xpos, ypos):
    global firstMouse, cameraFront, yaw, pitch, lastX, lastY
    if firstMouse:
        lastX = xpos
        lastY = ypos
        firstMouse = False

    xoffset = xpos - lastX
    yoffset = lastY - ypos
    lastX = xpos
    lastY = ypos

    sensitivity = 0.3 
    xoffset *= sensitivity
    yoffset *= sensitivity

    yaw += xoffset;
    pitch += yoffset;

    if pitch >= 90.0: pitch = 90.0
    if pitch <= -90.0: pitch = -90.0

    front = glm.vec3()
    front.x = math.cos(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    front.y = math.sin(glm.radians(pitch))
    front.z = math.sin(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    cameraFront = glm.normalize(front)
    
glfw.set_key_callback(window,key_event)
glfw.set_cursor_pos_callback(window, mouse_event)
glfw.set_input_mode(window, glfw.CURSOR, glfw.CURSOR_DISABLED) # Trava e esconde o mouse na janela

### Matrizes *Model*, *View*, e *Projection*
Teremos uma aula específica para entender o seu funcionamento.

In [21]:
def model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z):
    
    angle = math.radians(angle)
    
    matrix_transform = glm.mat4(1.0) # Instanciando uma matriz identidade
    
    # Aplicando rotacao
    matrix_transform = glm.rotate(matrix_transform, angle, glm.vec3(r_x, r_y, r_z))
    
    # Aplicando translacao
    matrix_transform = glm.translate(matrix_transform, glm.vec3(t_x, t_y, t_z))    
    
    # Aplicando escala
    matrix_transform = glm.scale(matrix_transform, glm.vec3(s_x, s_y, s_z))
    
    matrix_transform = np.array(matrix_transform)
    
    return matrix_transform

def view():
    global cameraPos, cameraFront, cameraUp
    global cameraSpeed
    radius = 19.5
    
    # Verifica se ultrapassa o domo do ceu
    if(cameraPos[0]**2 + cameraPos[1]**2 + cameraPos[2]**2 > radius**2):
        if(cameraPos[0]<0.0):
            cameraPos[0] += 2*cameraSpeed
        else:
            cameraPos[0] -= 2*cameraSpeed
        
        if(cameraPos[1]<0.0):
            cameraPos[1] += 2*cameraSpeed
        else:
            cameraPos[1] -= 2*cameraSpeed
            
        if(cameraPos[2]<0.0):
            cameraPos[2] += 2*cameraSpeed
        else:
            cameraPos[2] -= 2*cameraSpeed
            
    # Verifica se ultrapassa o chao
    if(cameraPos[1] < 0.35):
        cameraPos[1] = 0.35
            
    mat_view = glm.lookAt(cameraPos, cameraPos + cameraFront, cameraUp);
    mat_view = np.array(mat_view)
    return mat_view

def projection():
    global altura, largura
    # perspective parameters: fovy, aspect, near, far
    mat_projection = glm.perspective(glm.radians(45.0), largura/altura, 0.1, 1000.0)
    mat_projection = np.array(mat_projection)    
    return mat_projection

Nesse momento, exibimos a janela.

In [22]:
glfw.show_window(window)
glfw.set_cursor_pos(window, lastX, lastY)

### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que acontecem algumas interações com o `OpenGL`.

In [23]:
glEnable(GL_DEPTH_TEST) # Importante para 3D
   
ang = 0.1 # Ângulo de rotação do Sol
rotation = 0.0 # Ângulo de rotação do cachorro
    
while not glfw.window_should_close(window):

    glfw.poll_events() 
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glClearColor(0.2, 0.2, 0.2, 1.0)
    
    if polygonal_mode==True:
        glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    if polygonal_mode==False:
        glPolygonMode(GL_FRONT_AND_BACK,GL_FILL)
    
    
    # Desenhando os objetos na janela
    
    draw_object(models_indexes[0], 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0)
    
    ang += 0.006
    draw_object(models_indexes[1], 0.0, 0.0, 1.0, 0.0, math.cos(ang)*17.0, math.sin(ang)*17.0, 0.0, 1.0, 1.0, 1.0, ka=1, kd=1, ks=1, ns=10000.0, emit_light = True)

    draw_object(models_indexes[2], 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0)
    draw_object(models_indexes[3], 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0)
    draw_object(models_indexes[4], 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0)
    
    draw_object(models_indexes[5], 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0)
    draw_object(models_indexes[6], 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0)
    draw_object(models_indexes[7], 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0)
    
    draw_object(models_indexes[8], 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0)
    draw_object(models_indexes[9], 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0)
    
    rotation += 0.5
    draw_object(models_indexes[10], rotation, 0.0, 1.0, 0.0, -3.0, 1.4, 0.0, 0.1, 0.1, 0.1)
    
    mat_view = view()
    loc_view = glGetUniformLocation(program, "view")
    glUniformMatrix4fv(loc_view, 1, GL_TRUE, mat_view)

    mat_projection = projection()
    loc_projection = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_projection, 1, GL_TRUE, mat_projection)    
    
    # Atualizando a posicao da camera/observador na GPU para calculo da reflexao especular
    loc_view_pos = glGetUniformLocation(program, "viewPos") # Recuperando localizacao da variavel viewPos na GPU
    glUniform3f(loc_view_pos, cameraPos[0], cameraPos[1], cameraPos[2]) # Posicao da camera/observador (x,y,z)
    
    glfw.swap_buffers(window)

glfw.terminate()